## Bond Valuation - Pricing Technique

A bond is valued as how investors perceives to obtain a certain yield.  Hence, the price of a bond is simply the net present value of all future cash flows discounted back to today based on an internal rate of return called _yield to maturity_.

For a bond that pays annual coupon:
$$
PV = \frac{C}{(1 + r)^1} + \frac{C}{(1 + r)^2} + ... + \frac{C + P}{(1 + r)^n}
$$

where 
$C$ = coupon, 
$P$ = principal,
$r$ = yield to maturity,
$n$ = number of periods

To calculate $PV$ for a bond with semi-annual coupon (coupon frequency = 2), several adjustments must be made:

* $C$ is divided by 2 
* $r$ is divided by 2
* $n$ is multiplied by 2 (so we have twice the number of terms than an annual coupon bond)

The $\frac{1}{(1 + r)^t}$ term in the above formula is called _discount factor_.

**Task**

1. Enhance the function `payment_schedule(bond::Bond, as_of_date::Date, yield::Float64)` to accept yield as a new argument.  Now, it must return the following columns:

  - `date`: date of the cash flow
  - `amount`: coupon payment, or coupon+principal for the last payment
  - `days`: number of days between the as of date and the payment date
  - (new) `discount_factor`: discount factor for the period
  - (new) `value`: present value of the cash flow, which is simply amount multipled by discount factor

2. Write a new function `price(bond::Bond, as_of_date::Date, yield::Float64)` that returns the price of the bond, as calculated below. You can assume par amount of \\$1,000 and round the result to 8 decimal places.

$$
Price = \frac{PV}{ParAmount} * 100
$$

Test your code with the following:

```julia
bond = Bond("9128284V9", 0.02875, Date(2028, 8, 15), 2)
as_of_date = Date(2018, 8, 15)
@show payment_schedule(bond, as_of_date, 0.02875)
@show price(bond, as_of_date, 0.02875)
```

The result should look like:
```
│ Row │ date       │ amount  │ days  │ discount_factor │ value   │
├─────┼────────────┼─────────┼───────┼─────────────────┼─────────┤
│ 1   │ 2019-02-15 │ 14.375  │ 184   │ 0.985829        │ 14.1713 │
│ 2   │ 2019-08-15 │ 14.375  │ 365   │ 0.971858        │ 13.9705 │
│ 3   │ 2020-02-15 │ 14.375  │ 549   │ 0.958086        │ 13.7725 │
...
price(bond, as_of_date, 0.02875) = 100.0
```

## Bond Valuation 3 - Full Price

We had a problem in yesterday's implementation.  The formula assumes that the valuation date falls on a coupon date (or the issue date) because we discount the cash flows using full periods (1, 2, ..., $n$).  In practice, the valuation date is likely sometime in between coupon dates.

To make it correct, we must adjust the discount factors to reflect partial periods.  

First, let's define
$$
w = \frac{DaysFromValuationDateToNextCouponDate}{DaysInCouponPeriod}
$$

Then, the exponents for the $PV$ formula is adjusted as follows:
$$
PV = \frac{C}{(1 + r)^{w}} + \frac{C}{(1 + r)^{1 + w}} + ... + \frac{C + P}{(1 + r)^{n - 1 + w}}
$$


**The Task**

Enhance your `payment_schedule` function to include `period` column and adjust `discount_factor` properly.  Test your code with the following bond.

```
bond = Bond("9128284V9", 0.02875, 2028-08-15, 2)

as_of_date = Date(2018, 11, 22) 

cash_flow(bond, as_of_date, 0.03067) = 20×6 DataFrame
│ Row │ date       │ amount  │ days  │ discount_factor │ value   │ period   │
├─────┼────────────┼─────────┼───────┼─────────────────┼─────────┼──────────┤
│ 1   │ 2019-02-15 │ 14.375  │ 85    │ 0.992994        │ 14.2743 │ 0.461957 │
│ 2   │ 2019-08-15 │ 14.375  │ 266   │ 0.977997        │ 14.0587 │ 1.46196  │
│ 3   │ 2020-02-15 │ 14.375  │ 450   │ 0.963226        │ 13.8464 │ 2.46196  │
...
```
